In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm

In [ ]:
url = """
https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&
search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&
pageNumber=1&sortDirection=false&recordsPerPage=_50&showLotsInfoHidden=false&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&
currencyIdGeneral=-1&okpd2IdsWithNested=on&okpd2Ids=8873937%2C8873938&
okpd2IdsCodes=62%2C63&gws=%D0%92%D1%8B%D0%B1%D0%B5%D1%80%D0%B8%D1%82%D0%B5+%D1%82%D0%B8%D0%BF+%D0%B7%D0%B0%D0%BA%D1%83%D0%BF%D0%BA%D0%B8&
OrderPlacementSmallBusinessSubject=on&OrderPlacementRnpData=on&OrderPlacementExecutionRequirement=on&orderPlacement94_0=0&orderPlacement94_1=0&orderPlacement94_2=0
"""

# инициализация браузера с автоматическим закрытием
with webdriver.Safari() as browser:
    # переместить окно
    # browser.set_window_position(0, -1000)
    # раскрыть на весь экран
    browser.maximize_window()
    
    # открыть сайт
    browser.get(url)
    time.sleep(2)

    data = []
    
    # находим элемент с блоком заказов
    el_block = browser.find_element(By.CLASS_NAME, 'search-registry-entrys-block')
    
    # находим в этом блоке все заказы
    all_zak = el_block.find_elements(By.CLASS_NAME, 'search-registry-entry-block')
    
    # циклимся по списку для парсинга элементов
    for zak in tqdm(all_zak):
        element = {}
        
        # ищем элементы
        num = zak.find_element(By.CLASS_NAME, 'registry-entry__header-mid__number')
        num = num.find_element(By.TAG_NAME, 'a')
        order_url = num.get_attribute('href')
        obj_zak = zak.find_element(By.CLASS_NAME, 'registry-entry__body-value').text
        price = zak.find_element(By.CLASS_NAME, 'price-block__value').text
        customer = zak.find_element(By.CLASS_NAME, 'registry-entry__body-href')
        customer_name = customer.text
        customer_url = customer.find_element(By.TAG_NAME, 'a').get_attribute('href')
        
        # формируем словарь с данными
        element['order_id'] = num.text.strip()
        element['obj_zak'] = obj_zak.strip()
        element['price'] = price.strip()
        element['customer'] = customer_name.strip()
        element['order_url'] = order_url
        element['customer_url'] = customer_url
        
        data.append(element)
        
    # клик на следующую страницу с поиском кнопки через CSS
    next_page = browser.find_element(By.CSS_SELECTOR, "a.paginator-button")
    browser.execute_script("return arguments[0].scrollIntoView(true);", next_page)
    time.sleep(1)
    next_page.click()
    time.sleep(5)

    
print(len(data))

In [ ]:
data